In [1]:
import tensorflow as tf

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.metrics
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Training on C2 and Testing on C2

In [3]:
NS_case3 = pd.read_excel("../../NS_EOS_Case3.xlsx")
X_full = NS_case3.iloc[:, 0:7].values
y_full = NS_case3.iloc[:, [7,9,11]].values

from sklearn.cross_validation import train_test_split
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size = 1/6, random_state = 0)

#Standardizing the data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y = sc_y.fit_transform(y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Define the 2 HL model definition

In [4]:
def HL_2_model(i,j,result_new):
    #create model
    model = Sequential()  #creating sequential model; Dense specifies fully connected layer
    no_of_hidden_layers = 2
    node_in_each_layer = [i,j]
    model.add(Dense(node_in_each_layer[0],input_dim=7,activation='relu',kernel_initializer='normal')) #first hidden layer
    model.add(Dense(node_in_each_layer[1],activation='relu',kernel_initializer='normal')) #second hidden layer
    model.add(Dense(3,kernel_initializer='normal'))
    #compile model
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    #Train model
    train = model.fit(X,y,epochs=50,batch_size=16)  #size of dataset between 32-512
    #check performance
    y_pred = model.predict(X_test)

    y_pred = sc_y.inverse_transform(y_pred)
    #RMS value
    rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
    rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
    rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
    rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

    #R2 value
    r2 = sklearn.metrics.r2_score(y_test,y_pred)
    
    #Selection Criterion
    flag=0
    ext_dir = ''
    if rms<15 and r2>0.96:     
        flag = 1
        ext_dir = 'Good_models'
    else:
        flag=2
        ext_dir = 'Bad_models'
    result_new = result_new.append({'Model':'ANN(2HL-'+str(i)+','+str(j)+' Nodes)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                            'RMS_Lam':rms_lam, 'R2':r2, 'Flag':flag}, ignore_index=True)
    #create and open folder   
    dire = os.path.join('2HL',ext_dir,'HL_nodes-'+str(i)+'_'+str(j))
    if not os.path.exists(dire):
        os.mkdir(dire)
    plt.plot(train.history['loss'])
    plt.title("Training loss curve (50 epochs)")
    direc = os.path.join(dire,'Training loss curve (50 epochs)')
    plt.savefig(direc)
    plt.close()
    #Plotting
    titles = ['NS_Mass','NS_Rad','Lam']
    for i in range(3):
        rms = 'RMSE:'+str((np.sqrt(sklearn.metrics.mean_squared_error(y_test[:,i], y_pred[:,i]))))
        fig, ax = plt.subplots()
        ax.scatter(y_test[:,i], y_pred[:,i])
        ax.plot([y_test[:,i].min(), y_test[:,i].max()], [y_test[:,i].min(), y_test[:,i].max()], 'k--', lw=4)
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        plt.title(titles[i]+';ANN(2HL-'+str(i)+','+str(j)+' Nodes);[Case1+Case2 on Case3] \n'+rms)
        direc = os.path.join(dire,titles[i])
        plt.savefig(direc)
        plt.close()
    return result_new
    

In [5]:
result_new = pd.DataFrame(columns=['Model','RMS','RMS_NS_Mass','RMS_NS_Rad','RMS_Lam','R2','Flag'])
for i in range(1,21):
    for j in range(1,21):
        result_new = HL_2_model(i,j,result_new)
result_new

Train on 2232 samples
Epoch 1/50
2232/2232 [==============================] - 1s 434us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 2/50
2232/2232 [==============================] - 0s 76us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 3/50
2232/2232 [==============================] - 0s 64us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 4/50
2232/2232 [==============================] - 0s 62us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 5/50
2232/2232 [==============================] - 0s 60us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 6/50
2232/2232 [==============================] - 0s 63us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 7/50
2232/2232 [==============================] - 0s 61us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 8/50
2232/2232 [==============================] - 0s 59us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 9/50
2232/2232 [==============================] - 0

2232/2232 [==============================] - 0s 61us/sample - loss: 0.1852 - mean_squared_error: 0.1852
Epoch 22/50
2232/2232 [==============================] - 0s 67us/sample - loss: 0.1847 - mean_squared_error: 0.1847
Epoch 23/50
2232/2232 [==============================] - 0s 63us/sample - loss: 0.1847 - mean_squared_error: 0.1847
Epoch 24/50
2232/2232 [==============================] - 0s 61us/sample - loss: 0.1840 - mean_squared_error: 0.1840
Epoch 25/50
2232/2232 [==============================] - 0s 56us/sample - loss: 0.1840 - mean_squared_error: 0.1840
Epoch 26/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.1837 - mean_squared_error: 0.1837
Epoch 27/50
2232/2232 [==============================] - 0s 69us/sample - loss: 0.1833 - mean_squared_error: 0.1833
Epoch 28/50
2232/2232 [==============================] - 0s 65us/sample - loss: 0.1831 - mean_squared_error: 0.1831
Epoch 29/50
2232/2232 [==============================] - 0s 65us/sample - loss: 0.18

2232/2232 [==============================] - 0s 60us/sample - loss: 0.2106 - mean_squared_error: 0.2106
Epoch 41/50
2232/2232 [==============================] - 0s 60us/sample - loss: 0.2104 - mean_squared_error: 0.2104
Epoch 42/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.2101 - mean_squared_error: 0.2101
Epoch 43/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.2098 - mean_squared_error: 0.2098
Epoch 44/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.2098 - mean_squared_error: 0.2098
Epoch 45/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.2095 - mean_squared_error: 0.2095
Epoch 46/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.2094 - mean_squared_error: 0.2094
Epoch 47/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.2090 - mean_squared_error: 0.2090
Epoch 48/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.20

2232/2232 [==============================] - 0s 59us/sample - loss: 0.2092 - mean_squared_error: 0.2092
Epoch 11/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.2023 - mean_squared_error: 0.2023
Epoch 12/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 13/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.1942 - mean_squared_error: 0.1942
Epoch 14/50
2232/2232 [==============================] - 0s 57us/sample - loss: 0.1918 - mean_squared_error: 0.1918
Epoch 15/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.1902 - mean_squared_error: 0.1902
Epoch 16/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.1888 - mean_squared_error: 0.1888
Epoch 17/50
2232/2232 [==============================] - 0s 58us/sample - loss: 0.1876 - mean_squared_error: 0.1876
Epoch 18/50
2232/2232 [==============================] - 0s 61us/sample - loss: 0.18

2232/2232 [==============================] - 0s 62us/sample - loss: 0.2152 - mean_squared_error: 0.2152
Epoch 30/50
2232/2232 [==============================] - 0s 61us/sample - loss: 0.2146 - mean_squared_error: 0.2146
Epoch 31/50
2232/2232 [==============================] - 0s 66us/sample - loss: 0.2140 - mean_squared_error: 0.2140
Epoch 32/50
2232/2232 [==============================] - 0s 60us/sample - loss: 0.2133 - mean_squared_error: 0.2133
Epoch 33/50
2232/2232 [==============================] - 0s 60us/sample - loss: 0.2127 - mean_squared_error: 0.2127
Epoch 34/50
2232/2232 [==============================] - 0s 62us/sample - loss: 0.2123 - mean_squared_error: 0.2123
Epoch 35/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.2119 - mean_squared_error: 0.2119
Epoch 36/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.2117 - mean_squared_error: 0.2117
Epoch 37/50
2232/2232 [==============================] - 0s 59us/sample - loss: 0.21

...

2232/2232 [==============================] - 0s 76us/sample - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 26/50
2232/2232 [==============================] - 0s 65us/sample - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 27/50
2232/2232 [==============================] - 0s 71us/sample - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 28/50
2232/2232 [==============================] - 0s 67us/sample - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 29/50
2232/2232 [==============================] - 0s 73us/sample - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 30/50
2232/2232 [==============================] - 0s 94us/sample - loss: 0.0011 - mean_squared_error: 0.0011
Epoch 31/50
2232/2232 [==============================] - 0s 89us/sample - loss: 0.0011 - mean_squared_error: 0.0011
Epoch 32/50
2232/2232 [==============================] - 0s 74us/sample - loss: 0.0010 - mean_squared_error: 0.0010 
Epoch 33/50
2232/2232 [==============================] - 0s 66us/sample - loss: 9.8

2232/2232 [==============================] - 0s 74us/sample - loss: 6.4226e-04 - mean_squared_error: 6.4226e-04
Epoch 43/50
2232/2232 [==============================] - 0s 91us/sample - loss: 6.3898e-04 - mean_squared_error: 6.3898e-04
Epoch 44/50
2232/2232 [==============================] - 0s 64us/sample - loss: 6.0121e-04 - mean_squared_error: 6.0121e-04
Epoch 45/50
2232/2232 [==============================] - 0s 66us/sample - loss: 5.8703e-04 - mean_squared_error: 5.8703e-04
Epoch 46/50
2232/2232 [==============================] - 0s 62us/sample - loss: 5.8656e-04 - mean_squared_error: 5.8656e-04
Epoch 47/50
2232/2232 [==============================] - 0s 66us/sample - loss: 5.8493e-04 - mean_squared_error: 5.8493e-04
Epoch 48/50
2232/2232 [==============================] - 0s 63us/sample - loss: 6.5267e-04 - mean_squared_error: 6.5266e-04
Epoch 49/50
2232/2232 [==============================] - 0s 67us/sample - loss: 5.9268e-04 - mean_squared_error: 5.9268e-04
Epoch 50/50
2232/223

2232/2232 [==============================] - 0s 65us/sample - loss: 0.0108 - mean_squared_error: 0.0108
Epoch 10/50
2232/2232 [==============================] - 0s 64us/sample - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 11/50
2232/2232 [==============================] - 0s 65us/sample - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 12/50
2232/2232 [==============================] - 0s 63us/sample - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 13/50
2232/2232 [==============================] - 0s 64us/sample - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 14/50
2232/2232 [==============================] - 0s 63us/sample - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 15/50
2232/2232 [==============================] - 0s 65us/sample - loss: 0.0025 - mean_squared_error: 0.0025
Epoch 16/50
2232/2232 [==============================] - 0s 65us/sample - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 17/50
2232/2232 [==============================] - 0s 63us/sample - loss: 0.00

2232/2232 [==============================] - 0s 64us/sample - loss: 9.8720e-04 - mean_squared_error: 9.8720e-04
Epoch 29/50
2232/2232 [==============================] - 0s 64us/sample - loss: 9.8730e-04 - mean_squared_error: 9.8730e-04
Epoch 30/50
2232/2232 [==============================] - 0s 64us/sample - loss: 9.6342e-04 - mean_squared_error: 9.6342e-04
Epoch 31/50
2232/2232 [==============================] - 0s 65us/sample - loss: 8.9677e-04 - mean_squared_error: 8.9677e-04
Epoch 32/50
2232/2232 [==============================] - 0s 65us/sample - loss: 8.6100e-04 - mean_squared_error: 8.6100e-04
Epoch 33/50
2232/2232 [==============================] - 0s 63us/sample - loss: 8.6465e-04 - mean_squared_error: 8.6464e-04
Epoch 34/50
2232/2232 [==============================] - 0s 67us/sample - loss: 8.8293e-04 - mean_squared_error: 8.8293e-04
Epoch 35/50
2232/2232 [==============================] - 0s 64us/sample - loss: 8.3775e-04 - mean_squared_error: 8.3775e-04
Epoch 36/50
2232/223

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2,Flag
0,"ANN(2HL-1,1 Nodes)",67.551973,0.080361,0.650680,117.001612,-0.000546,2
1,"ANN(2HL-1,2 Nodes)",18.246672,0.047559,0.256117,31.603090,0.807206,2
2,"ANN(2HL-1,3 Nodes)",7.933333,0.057798,0.205739,13.739274,0.789579,2
3,"ANN(2HL-1,4 Nodes)",7.934908,0.058092,0.202320,13.742052,0.788918,2
4,"ANN(2HL-1,5 Nodes)",15.528169,0.041140,0.261285,26.894277,0.841204,2
5,"ANN(2HL-1,6 Nodes)",7.957367,0.058071,0.207233,13.780883,0.787428,2
6,"ANN(2HL-1,7 Nodes)",7.665588,0.057903,0.204913,13.275480,0.789524,2
7,"ANN(2HL-1,8 Nodes)",16.085218,0.040954,0.262157,27.859152,0.840346,2
8,"ANN(2HL-1,9 Nodes)",11.759159,0.036867,0.261005,20.365755,0.866047,2
9,"ANN(2HL-1,10 Nodes)",14.011155,0.039472,0.260003,24.266607,0.851950,2


In [12]:
#Segregating bad and good models
result_new_good = result_new[result_new['Flag']==1]
result_new_bad = result_new[result_new['Flag']==2]
result_new_v_good1 = result_new_good[result_new_good['R2']>0.999 ]  #Comment out later
result_new_v_good = result_new_v_good1[result_new_v_good1['RMS']<2]
result_new_v_good = result_new_v_good.reset_index()
result_new_v_good = result_new_v_good.drop(['index','Flag'],axis=1)
result_new_v_good

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,"ANN(2HL-14,15 Nodes)",1.788077,0.002016,0.024281,3.096944,0.999092
1,"ANN(2HL-14,20 Nodes)",1.772858,0.002171,0.021460,3.070605,0.999164
2,"ANN(2HL-15,14 Nodes)",1.749249,0.001905,0.025659,3.029679,0.999070
3,"ANN(2HL-15,15 Nodes)",1.753554,0.002343,0.020430,3.037175,0.999163
4,"ANN(2HL-15,18 Nodes)",1.685370,0.002387,0.018444,2.919087,0.999230
5,"ANN(2HL-15,19 Nodes)",1.806366,0.002069,0.020490,3.128650,0.999210
6,"ANN(2HL-16,12 Nodes)",1.922790,0.002572,0.020212,3.330307,0.999066
7,"ANN(2HL-16,15 Nodes)",1.833880,0.002490,0.018866,3.176316,0.999154
8,"ANN(2HL-16,18 Nodes)",1.734759,0.002733,0.020078,3.004623,0.999077
9,"ANN(2HL-16,19 Nodes)",1.500652,0.002097,0.023762,2.599095,0.999164


In [13]:
#Writing in file
direc1 = os.path.join('2HL','Bad_Model_details.xlsx')
result_new_bad.to_excel(direc1)
direc2 = os.path.join('2HL','Good_Model_details.xlsx')
result_new_good.to_excel(direc2)
direc3 = os.path.join('2HL','VGood_Model_details.xlsx')
result_new_v_good.to_excel(direc3)

In [14]:
list_of_V_Good_models = []
names = result_new_v_good['Model'].tolist()
for x in names:
    ij = x[8:-7].split(',')
    list_of_V_Good_models.append((int(ij[0]),int(ij[1])))
list_of_V_Good_models

[(14, 15),
 (14, 20),
 (15, 14),
 (15, 15),
 (15, 18),
 (15, 19),
 (16, 12),
 (16, 15),
 (16, 18),
 (16, 19),
 (16, 20),
 (17, 13),
 (17, 14),
 (17, 15),
 (17, 16),
 (17, 19),
 (18, 10),
 (18, 11),
 (18, 12),
 (18, 13),
 (18, 14),
 (18, 15),
 (18, 16),
 (18, 18),
 (18, 19),
 (19, 15),
 (19, 17),
 (19, 20),
 (20, 13),
 (20, 14),
 (20, 15),
 (20, 16),
 (20, 17),
 (20, 19),
 (20, 20)]

In [15]:
list_of_V_Good_models= [(14, 15),(14, 20),(15, 14),(15, 15),(15, 18),(15, 19),(16, 12),(16, 15),(16, 18),(16, 19),(16, 20),
                        (17, 13),(17, 14),(17, 15),(17, 16),(17, 19),(18, 10),(18, 11),(18, 12),(18, 13),(18, 14),(18, 15),
                        (18, 16),(18, 18),(18, 19),(19, 15),(19, 17),(19, 20),(20, 13),(20, 14),(20, 15),(20, 16),(20, 17),
                        (20, 19),(20, 20)]